# Tag 2. Kapitel 8. Daten-Manipulationen

## Lektion 44.  Guide zur Verwendung von tidyr

Wir kennen jetzt bereits `dplyr` und seine Funktionen. Zusätzlich können wir uns jetzt `tidyr` aneignen. Dies ist ein komplementäres Paket, das uns dabei hilft, ordentliche Datensätze zu erstellen. Was meinen wir überhaupt wenn wir "ordentlicher" Datensatz sagen?

Ordentliche Daten liegen vor, wenn jede Zeile eine eigene Beobachtung darstellt und jede Spalte eine Variable. Durch diese Organisation der Daten beinhaltet jede Zelle genau den Wert für eine Variable für eine Beobachtung. Durch diese Format können wir Daten leichter verstehen, analysieren und visualisieren.

Nach dieser Lektion könnt ihr euch auf folgendes Cheatsheet beziehen: [Data Wrangling](http://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf)

## Installation von tidyr

In [1]:
install.packages('tidyr',repos = 'http://cran.us.r-project.org')
# Evtl. data.table installieren
install.packages("data.table")

package 'tidyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\dm_78\AppData\Local\Temp\Rtmpu8x5pr\downloaded_packages
package 'data.table' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\dm_78\AppData\Local\Temp\Rtmpu8x5pr\downloaded_packages


In [3]:
library(tidyr)
library(data.table)

## Data.frames versus data.tables

Alle data.tables sind auch data.frames. Vereinfacht gesagt sind data.tables data.frames die extra Features bieten. 

data.frame ist in der Installation von R inbegriffen.

data.table ist ein Paket, dass data.frame erweitert. Zwei der wichtigsten Features sind Geschwindigkeit und saubere Syntax.

Jedoch ist die Syntax der data.table etwas unterschiedlich zur Standard R Syntax von data.frame. Diese Unterschiede können für ein untrainiertes Auge allerdings schwer zu erkennen sein. Wenn man mit einem Code-Abschnitt arbeitet, der nicht anzeigt, dass mit data.tables gearbeitet wird, und diesen auf einen Data Frame anwenden möchte, dann kann es zu Errors kommen.

Um einige der Unterschiede festzuhalten:

* viel schnellerer und sehr intuitive **by** Operationen
* Wir werden nicht ausversehen riesige Data Frames ausgeben, die wir mit Strg-C stoppen müssten. data.table verhindert solche Vorfälle
* schnellere und einfacheres Lesen von Datein mit **fread**
* das Paket beinhaltet weitere Funktionen wie %between% oder rbindlist
* und insgesamt einfach etwas schnellere Grundfunktionen

# Nutzung von tidyr

Wir werden einige der nützlichsten Funktionen in tidyr abdecken. Dazu gehören die folgenden:

* gather()
* spread()
* separate()
* unite()

## Beispiel Datensatz

Lass uns jetzt einige Daten erzeugen, die mit tidyr ordentlich gemacht werden müssen

In [6]:
firma <- c(1,1,1,2,2,2,3,3,3)
jr <- c(1998,1999,2000,1998,1999,2000,1998,1999,2000)
q1 <- runif(9, min=0, max=100)
q2 <- runif(9, min=0, max=100)
q3 <- runif(9, min=0, max=100)
q4 <- runif(9, min=0, max=100)

df <- data.frame(firma=firma,jahr=jr,Qtr1 = q1,Qtr2 = q2,Qtr3 = q3,Qtr4 = q4)

In [7]:
df

firma,jahr,Qtr1,Qtr2,Qtr3,Qtr4
1,1998,87.47157,15.28078,23.571765,7.014554
1,1999,27.29214,83.72309,5.300331,30.810351
1,2000,95.05862,28.71396,45.258085,1.124289
2,1998,73.14976,39.34447,1.585104,98.313194
2,1999,16.64702,18.77766,20.727349,16.766125
2,2000,88.94860,92.46872,85.425926,25.821003
3,1998,82.33251,18.43848,72.428504,30.069440
3,1999,36.98951,74.12204,70.804638,46.723383
3,2000,19.79196,41.06764,44.210498,68.088605


# Gather() und Spread()

Manchmal vergleichen Leute diese Operationen mit Pivot-Tabellen in Excel. Schauen wir uns einige Beispiele an:

## gather()

Die `gather()` Funktion fasst mehrere Spalten in Schlüssel-Paar Werte zusammen. Der Data Frame von oben würde als "breit" bezeichnet werden, da jedes Quartal eine eigene Spalte bzw. Variable hat. Um diese Zeitdimension neu zu ordnen, können wir alle Quartale in eine Spalte vereinen. Außerdem können wir dann alle Werte, die mit den einzelnen Quartalen asoziiert sind, in eine zweite Spalte schreiben.

In [9]:
# Unter Verwendung des Pipe Operators
df %>% gather(Quartal,Umsatz,Qtr1:Qtr4) %>% head

firma,jahr,Quartal,Umsatz
1,1998,Qtr1,87.47157
1,1999,Qtr1,27.29214
1,2000,Qtr1,95.05862
2,1998,Qtr1,73.14976
2,1999,Qtr1,16.64702
2,2000,Qtr1,88.94860


In [10]:
# Nur mit der Funktion
head(gather(df,Quartal,Umsatz,Qtr1:Qtr4))

firma,jahr,Quartal,Umsatz
1,1998,Qtr1,87.47157
1,1999,Qtr1,27.29214
1,2000,Qtr1,95.05862
2,1998,Qtr1,73.14976
2,1999,Qtr1,16.64702
2,2000,Qtr1,88.94860


## spread()

Ist das Gegenstück zu `gather()`:

In [15]:
# Step 1.
aktien <- data.frame(
  datum = as.Date('2009-01-01') + 0:9,
  GOOG = rnorm(10, 0, 1),
  TSLA = rnorm(10, 0, 2),
  MSFT = rnorm(10, 0, 4)
)
aktien

datum,GOOG,TSLA,MSFT
2009-01-01,-0.97225318,2.32337025,-1.9963137
2009-01-02,0.13095594,-2.49209090,0.6191049
2009-01-03,-0.62116557,-2.73842573,-4.8257505
2009-01-04,-1.28596389,-0.83800287,7.0864154
2009-01-05,0.77076266,-1.33986971,-6.0641254
2009-01-06,1.39330964,-0.36729839,-3.4876813
2009-01-07,-1.35689468,1.08151492,-4.0087587
2009-01-08,0.05177523,-1.73762695,2.1686812
2009-01-09,-0.73093871,0.08815671,5.5125539
2009-01-10,1.81492124,1.85688997,4.7441420


In [16]:
# Step 2.
# Daten als Key-Value Paare darstellen
aktienm <- aktien %>% gather("aktie", "preis", -datum)
aktienm # 30 x 3 Datensatz

datum,aktie,preis
2009-01-01,GOOG,-0.97225318
2009-01-02,GOOG,0.13095594
2009-01-03,GOOG,-0.62116557
2009-01-04,GOOG,-1.28596389
2009-01-05,GOOG,0.77076266
2009-01-06,GOOG,1.39330964
2009-01-07,GOOG,-1.35689468
2009-01-08,GOOG,0.05177523
2009-01-09,GOOG,-0.73093871
2009-01-10,GOOG,1.81492124


In [17]:
# Step 3.
# Key-Value Datensätze über Aktie darstellen
aktienm %>% spread("aktie", "preis")

datum,GOOG,MSFT,TSLA
2009-01-01,-0.97225318,-1.9963137,2.32337025
2009-01-02,0.13095594,0.6191049,-2.49209090
2009-01-03,-0.62116557,-4.8257505,-2.73842573
2009-01-04,-1.28596389,7.0864154,-0.83800287
2009-01-05,0.77076266,-6.0641254,-1.33986971
2009-01-06,1.39330964,-3.4876813,-0.36729839
2009-01-07,-1.35689468,-4.0087587,1.08151492
2009-01-08,0.05177523,2.1686812,-1.73762695
2009-01-09,-0.73093871,5.5125539,0.08815671
2009-01-10,1.81492124,4.7441420,1.85688997


In [18]:
# Step 3.
# Key-Value Datensätze über Datum darstellen
aktienm %>% spread("datum", "preis")

aktie,2009-01-01,2009-01-02,2009-01-03,2009-01-04,2009-01-05,2009-01-06,2009-01-07,2009-01-08,2009-01-09,2009-01-10
GOOG,-0.9722532,0.1309559,-0.6211656,-1.2859639,0.7707627,1.3933096,-1.356895,0.05177523,-0.73093871,1.814921
MSFT,-1.9963137,0.6191049,-4.8257505,7.0864154,-6.0641254,-3.4876813,-4.008759,2.16868124,5.51255385,4.744142
TSLA,2.3233703,-2.4920909,-2.7384257,-0.8380029,-1.3398697,-0.3672984,1.081515,-1.73762695,0.08815671,1.856890


# Separate und Unite

## separate()

Ausgehend von einer Regular Expression oder einem Vektor von Zeichenpositionen kann `separate()` eine einzelne Spalte in mehrere aufteilen:

In [34]:
v <- c(NA, "a_._x_j", "b_y", "c_z_w")
df <- data.frame(v)
v
df

[1] NA        "a_._x_j" "b_y"     "c_z_w"

v
NA
a_._x_j
b_y
c_z_w


In [35]:
help(separate)

In [36]:
df %>% separate(v, c("col1", "col2", "col3"))

Warning message:
"Expected 3 pieces. Missing pieces filled with `NA` in 1 rows [3]."

col1,col2,col3
NA,NA,NA
a,x,j
b,y,NA
c,z,w


## unite()

`unite()` ist üblicherweise die Funktion um mehrere Spalten zu einer zusammenzufassen:

In [21]:
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


In [37]:
mtcars %>%
  unite(vs.am, sep=".", vs, am) %>% head

,mpg,cyl,disp,hp,drat,wt,qsec,vs.am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0.1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0.1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1.1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1.0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0.0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1.0,3,1


In [25]:
help(unite)

In [38]:
# separate() ist das Gegenstück zu unite()
# vergleichen Sie 
mtcars %>%
  unite(vs_am, vs, am) %>%
  separate(vs_am, c("vs", "am")) %>% head

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


In [39]:
# ... mit mtcars
mtcars %>% head

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


Herzlichen Glückwunsch! Sie sind mit Lektion 44. fertig!